In [2]:
%load_ext Cython
%matplotlib inline
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [7]:
import numpy as np

a = np.arange(1,16).reshape(3,5)
print(a)


[[ 1  2  3  4  5]
 [ 6  7  8  9 10]
 [11 12 13 14 15]]


In [8]:
np.linalg.svd(a)

(array([[-0.20166491,  0.89031713,  0.40824829],
        [-0.5168305 ,  0.25733163, -0.81649658],
        [-0.83199609, -0.37565388,  0.40824829]]),
 array([3.51272233e+01, 2.46539670e+00, 1.58312250e-15]),
 array([[-0.35455706, -0.39869637, -0.44283568, -0.486975  , -0.53111431],
        [-0.68868664, -0.37555453, -0.06242242,  0.2507097 ,  0.56384181],
        [ 0.56979546, -0.33522256, -0.62629151, -0.02093113,  0.41264974],
        [-0.13260267,  0.29037552, -0.52552174,  0.71032761, -0.34257871],
        [-0.24031161,  0.70944196, -0.36274763, -0.44158417,  0.33520145]]))

In [13]:
%%cython
# distutils: language = c++
# distutils: sources = power_method.cpp
# cython: language_level = 3
# cython: boundscheck = False
# cython: wraparound = False

from libcpp.vector cimport vector

cdef extern from "power_method.cpp":
    struct usv_native:
        const vector[vector[double]] U
        const vector[double] S
        const vector[vector[double]] V
        
    void free_svd_result(usv_native* r)
    
    usv_native power_method_with_deflation()
